In [64]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Lambda, LSTM, RepeatVector, TimeDistributed, Flatten
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.losses import binary_crossentropy
from keras import backend as K

In [65]:
# Load the original dataset
processed_data = pd.read_csv('Processed Data/Aruba_17/processed_data.csv')
# only use the first 1000 rows
processed_data = processed_data.head(3200)
# Extract the relevant columns from the dataset
timestamp = processed_data['Timestamp'].values
device_id = processed_data['Device ID'].values
status = processed_data['Status'].values
activity = processed_data['Activity'].values
activity_status = processed_data['Activity Status'].values

# Prepare the data for input into the VAE model
X = np.stack((timestamp, device_id, status, activity, activity_status), axis=1)

# Normalize the data using minMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
batch_size = 32
validation_split = 0.2
timesteps = 32 # number of previous records considered
input_dim = X.shape[1] # number of features, there are 5 features in the dataset
num_classes = 14 # number of patters we need to recognize "large scale"

X_train, X_val, y_train, y_val = train_test_split(X, X, test_size=validation_split, shuffle=False)
X_train = X_train.reshape((int(X_train.shape[0]/timesteps), timesteps, input_dim))
X_val = X_val.reshape((int(X_val.shape[0]/timesteps), timesteps, input_dim))
# you need to reshape the y_train and y_val as well
y_train = y_train.reshape((int(y_train.shape[0]/timesteps), timesteps, input_dim))
y_val = y_val.reshape((int(y_val.shape[0]/timesteps), timesteps, input_dim))

latent_dim = 2
encoding_dim = 32

# Set the input shape for the VAE model
inputs = Input(batch_shape=(batch_size, timesteps, input_dim), name='encoder_input')
x = LSTM(encoding_dim, return_sequences=True)(inputs) # Add LSTM layer with return_sequences set to True
x = LSTM(encoding_dim, return_sequences=False)(x) # Add another LSTM layer with return_sequences set to False
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
# encoder.summary()

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(encoding_dim, activation='relu')(latent_inputs)
x = RepeatVector(timesteps)(x) # Repeat the hidden state timesteps times
x = LSTM(encoding_dim, return_sequences=True)(x) # Add a LSTM layer with return_sequences set to True
x = LSTM(encoding_dim, return_sequences=True)(x) # Add another LSTM layer with return_sequences set to True
x = TimeDistributed(Dense(input_dim, activation='sigmoid'))(x)

# Add fully connected layer to classify sequences into 14 categories
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(14, activation='softmax')(x)

outputs = x
decoder = Model(latent_inputs, outputs, name='decoder')
# decoder.summary()

outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

reconstruction_loss = binary_crossentropy(inputs, outputs)
reconstruction_loss *= input_dim * timesteps
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
kl_loss = tf.keras.backend.expand_dims(kl_loss, axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
# vae.summary()

batch_size = 32
num_samples = X_train.shape[0]
steps_per_epoch = num_samples // batch_size
history = vae.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_data=(X_val, y_val), steps_per_epoch=steps_per_epoch)

# Generate a fake dataset using the VAE model

n_samples = len(processed_data)
noise = np.random.normal(size=(n_samples, 5 - latent_dim))
noise = np.concatenate([noise, np.zeros((n_samples, latent_dim))], axis=-1)
# reshape noise to have the correct shape
noise = noise.reshape((int(noise.shape[0]/timesteps), timesteps, input_dim))
predicted_values = vae.predict(noise, batch_size=batch_size)

predicted_values = predicted_values.reshape((predicted_values.shape[0] * predicted_values.shape[1], predicted_values.shape[2]))

# undo the normalization
predicted_values = scaler.inverse_transform(predicted_values)

# Round each of the values in the array to the nearest integer
predicted_values = np.rint(predicted_values)

# Save the prediction data to a new file 'predicted_Data.csv'
predicted_data = pd.DataFrame(predicted_values, columns=['Timestamp', 'Device ID', 'Status', 'Activity', 'Activity Status'])
predicted_data.to_csv('Predictions/Aruba_17_prediction.csv', index=False)


ValueError: Exception encountered when calling layer "tf.keras.metrics.binary_crossentropy_5" (type TFOpLambda).

Dimensions must be equal, but are 5 and 14 for '{{node tf.keras.metrics.binary_crossentropy_5/mul}} = Mul[T=DT_FLOAT](Placeholder, tf.keras.metrics.binary_crossentropy_5/Log)' with input shapes: [32,32,5], [32,14].

Call arguments received by layer "tf.keras.metrics.binary_crossentropy_5" (type TFOpLambda):
  • y_true=tf.Tensor(shape=(32, 32, 5), dtype=float32)
  • y_pred=tf.Tensor(shape=(32, 14), dtype=float32)
  • from_logits=False
  • label_smoothing=0.0
  • axis=-1